In [1]:
import numpy as np  # The Numpy numerical computing library
import pandas as pd  # The Pandas data science library
import requests  # The requests library for HTTP requests in Python
import xlsxwriter  # The XlsxWriter libarary for
import math  # The Python math module
import pprint

# Function sourced from
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]


stocks = pd.read_csv("sp_500_stocks.csv")

In [3]:

from secrets import IEX_CLOUD_API_TOKEN

my_columns = ["Ticker", "Price", "Market Capitalization", "Number Of Shares to Buy"]
final_dataframe = pd.DataFrame(columns=my_columns)

symbol_groups = list(chunks(stocks["Ticker"], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))

for symbol_string in symbol_strings:
    #     print(symbol_strings)
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        new_dataframe = pd.DataFrame(
            [
                [
                    symbol,
                    data[symbol]["quote"]["latestPrice"],
                    data[symbol]["quote"]["marketCap"],
                    "N/A",
                ]
            ],
            columns=my_columns,
        )
        final_dataframe = pd.concat((final_dataframe, new_dataframe),ignore_index=True)
#final_dataframe = pd.concat((final_dataframe, new_dataframe), axis=0)

print(final_dataframe)

    Ticker   Price Market Capitalization Number Of Shares to Buy
0        A  136.67           40739723655                     N/A
1      AAL    15.1            9603544555                     N/A
2      AAP  209.76           12976529072                     N/A
3     AAPL  171.12         2697052111768                     N/A
4     ABBV  151.01          272231375516                     N/A
..     ...     ...                   ...                     ...
498    YUM  121.46           35648583784                     N/A
499    ZBH  122.79           25776282352                     N/A
500   ZBRA  408.89           21828265288                     N/A
501   ZION   66.86           10192713712                     N/A
502    ZTS  206.09           96573995215                     N/A

[503 rows x 4 columns]


In [22]:
portfolio_size = input("Enter the value of your portfolio:")
position_size = float(portfolio_size) / len(final_dataframe.index)

In [26]:
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size/final_dataframe['Price'][i])

In [28]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [29]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [30]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

writer.save()